### 1) This question is about using a webtool, without the need for any written answers.

### 2) Draw an ANN using the original artificial neurons (like the ones in Figure 10-3) that computes A ⊕ B = (A ∧ ¬B) ∨ (¬ A V B).

Done on paper

### 3) Why is it generally prefrable to use a logistic regression classifier rather than a classic perceptron (i.e., a single layer of threshold logic units trained using the perceptron training algorithm)? How can you tweak a perceptron to make it equivalent to a logistic regression classifier?

Because a common perceptron supposes linearly separable datasets. That is, if the decision boundary is not linear, than a classic perceptron won't be able to correctlt classify instances, whereas a logistic regression could potentially do. If you change the output activation function to a sigmoid, and train it with gradient descent or other equivalent method, than it will converge to the same result as in a usual logistic regression, with class probabilities.

### 4) Why was the sigmoid activation function a key ingredient in training the first MLPs?

Because the sigmoid's derivative is always non-zero, gradient descent methods will keep iterating and converging to global minimum, whereas other activation functions, like step functions, where the derivative is always equal to zero, it cannot converge to global minimum

### 5) Name three popular activation functions. Can you draw them?

Relu is 0 for values lower than 0 45 degrees linear for values higher than 0 (y = 0 if x <= 0; y = x if x >0).

Sigmoid is a S-shaped curve that takes values from -infinity to +infinity and maps then to (0,1)

Linear activation functions make a linear transformation on the inputs, mapping then to a single output.

### 6) Suppose you have an MLP composed of one input layer with 10 passthough neurons, followed by one hidden layer with 50 artificial neurons, and finally one output layer with 3 artificial neurons. All artificial neurons use the ReLU activation function.

a) What is the shape of the input matrix $\textbf{X}$?

Suppose you have m instances. Then $\textbf{X}$ is mx10.

b) What are the shapes of the hidden layer's weight matrix $\textbf{W}_h$ and bians vector $\textbf{b}_h$?

Each hidden layer is connected to 10 neurons, where each column contains the 10 weights of a single neuron, so $\textbf{W}_h$ is 10x50.

Since we have 1 bias for each neuron, our vector $\textbf{b}_h$ has size 50.


c) What are the shapes of the output layer's weight matrix $\textbf{W}_0$ and bias vector $\textbf{b}_0$?

Each output layer is connected to 50 neuros, so $\textbf{W}_0$ has size 50x3. Since we have 3 neuros on the output layer, $\textbf{b}_0$ has size 3.

d) What is the shape of the network's output matrix $\textbf{Y}$?

Again, suppposing we have m instances, then $\textbf{Y}$ is mx3


e) Write the equation that computed the newtork's output matrix $\textbf{Y}$ as a function of $\textbf{X}$, $\textbf{W}_h$, $\textbf{b}_h$, $\textbf{W}_0$ and $\textbf{b}_0$

$Y = ReLU(ReLU(\textbf{XW}_h + \textbf{b}_h)\textbf{W}_0+ \textbf{b}_0)$

### 7) How many neurons do you need in the output layer if you want to classify email into spam or ham? What activation funciton shoud you use in the output layer? If instead you want to tackle MNIST, how many neuros do you need i the output layer, and which activation function should you use? What about for getting your network to predict housing prices, as in Chapter 2?

To classify an email into either spam or ham, you can use a single-neuron output layer with a sigmoid activation function. In the MNIST case, where your classification isn't binary, you would need 10 neurons, one for each possible output, with the softmax activation function. For housing prices prediction, where the output is continuous, you should use one neuron with a linear activation function.

### 8) What is backpropagation and how does it work? What is the difference between backpropagation and revese-mode autodiff?

Backpropagation is one way to train MLPs (and possibly other deep learning algorithms), that is, to find the weights that best fit the data to sample. It starts by computing the estimates with the weigths from previous iteration (or random weights in the first one). Then, for the output layer, it calculates the derivative of the loss function with respect to each previous neuron, and from each previous neuron to the ones immediately prior and so one until the first layer. Then, with the chain rule, it is able to calculate the derivative of the loss function with respect to each weight from prior neurons (i.e., calculate the gradient of the loss function), and update these weights in a gradient descent step. It repeats until convergence. Reverse-mode autodiff is a technique used in backpropagation to calculate the gradients.

### 9) Can you list all the hyperparameters you can tweak in a basic MLP? If the MLP overfits the training data, how could you tweak these hyperparameter to try to solve the problem?

Hyperparameters and tweaks to decrease overfiting:

- Number of hidden layers: decrease

- Number of neurons per hidden layer: decrease

- Learning rate:

- Optimizer method:

- Activation Funtion:

- Batch Size:

- Number of iterations:

### 10) Train a deep MLP on the MNIST dataset (you can load it using tf.keras.datasets.mnist.load_data()). See if you can get over 98% accuracy by manually tuning the hyperparameters. Try searching for the optimal learning rate by using the approach presented in this chapter (i.e., by growing the learning rate exponentially, plotting the loss, and finding the point where the loss shoots up). Next, try tuning the hyperparameters using Keras Tuner with all the bells and whistles - save checkpoints, use early stopping, and plot learning curves using TensorBoard.

In [2]:
import tensorflow as tf

In [3]:
#Import data
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [36]:
#Create and compile the model
tf.random.set_seed(10)
input_layer = tf.keras.layers.Input(shape = X_train.shape[1:])
flat_layer = tf.keras.layers.Flatten()(input_layer)
hidden1 = tf.keras.layers.Dense(300, activation = "relu")(flat_layer)
hidden2 = tf.keras.layers.Dense(300, activation = "relu")(hidden1)
hidden3 = tf.keras.layers.Dense(300, activation = "relu")(hidden2)
hidden4 = tf.keras.layers.Dense(300, activation = "relu")(hidden3)
output_layer = tf.keras.layers.Dense(10, activation = "softmax")(hidden4)
model = tf.keras.Model(inputs = [input_layer], outputs = [output_layer])

optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-4)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer,
              metrics = ["accuracy"])

model.summary()

#Callbacks
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience = 10,
                                                     restore_best_weights = True)

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)          │ (None, 28, 28)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_10 (Flatten)                 │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 300)                 │         235,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_38 (Dense)                     │ (None, 300)                 │          90,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_39 (Dense)                     │ (None, 300)                 │          90,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_40 (Dense)                     │ (None, 300)                 │          90,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_41 (Dense)                     │ (None, 10)                  │           3,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 509,410 (1.94 MB)

 Trainable params: 509,410 (1.94 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
#Train
history = model.fit(X_train, y_train, epochs = 50,
                    validation_data = (X_valid, y_valid),
                    callbacks = [early_stopping_cb])

Epoch 1/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8272 - loss: 2.0706 - val_accuracy: 0.9404 - val_loss: 0.2145
Epoch 2/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9434 - loss: 0.1997 - val_accuracy: 0.9586 - val_loss: 0.1471
Epoch 3/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9553 - loss: 0.1589 - val_accuracy: 0.9660 - val_loss: 0.1191
Epoch 4/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9630 - loss: 0.1273 - val_accuracy: 0.9650 - val_loss: 0.1377
Epoch 5/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9680 - loss: 0.1083 - val_accuracy: 0.9688 - val_loss: 0.1233
Epoch 6/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9756 - loss: 0.0833 - val_accuracy: 0.9676 - val_loss: 0.1211
Epoch 7/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9779 - loss: 0.0734 - val_accuracy: 0.9702 - val_loss: 0.1156
Epoch 8/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9804 - loss: 0.0683 - 

In [40]:
#Test
model.evaluate(X_valid, y_valid)

157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9799 - loss: 0.0881


[0.09446649253368378, 0.980400025844574]

We got over 98% accuracy on the validation set